In [2]:
import os,sys
import re
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
from glob import glob

In [3]:
cvdf = pd.read_csv("csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv")
cvdf.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20
0,NaN,Thailand,15.0000,101.0000,2,3,5,7,8,8,...,75,82,114,147,177,212,272,322,411,599
1,NaN,Japan,36.0000,138.0000,2,1,2,2,4,4,...,701,773,839,825,878,889,924,963,1007,1086
2,NaN,Singapore,1.2833,103.8333,0,1,3,3,4,5,...,200,212,226,243,266,313,345,385,432,455
3,NaN,Nepal,28.1667,84.2500,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,2
4,NaN,Malaysia,2.5000,112.5000,0,0,0,3,4,4,...,197,238,428,566,673,790,900,1030,1183,1306


In [4]:
monthdict={1:0,2:31,3:60}
for i,li in enumerate(cvdf.columns):
    if i>=4:
        tmp=li.split("/")
        day=monthdict[int(tmp[0])]+int(tmp[1])
        day=day-22 
        cvdf=cvdf.rename(columns={li: day}, errors="raise")
cvdf=cvdf.sort_values(by=['Country/Region'])
cvdf=cvdf.reset_index(drop=True)
cvdf.drop("Lat", axis=1, inplace=True)
cvdf.drop("Long", axis=1, inplace=True)
cvdf.to_csv("covidtimeall.csv",index=False)

In [5]:
df = pd.read_csv("covidtimeall.csv")
df.head()
workaslist=list(df.iloc)
#plt.ylim(0,500)
countryid={}
for i,wl in enumerate(workaslist):
    y = list(wl[2:])
    leg = wl["Country/Region"]
    if not countryid.keys():
        countryid[leg]=y
    elif countryid.keys() and leg not in countryid.keys():    
        countryid[leg]=y
    elif countryid.keys() and leg in countryid.keys(): 
        tmp = zip(countryid[leg],y)
        utmp = [sum(item) for item in tmp]
        countryid[leg]=utmp

In [6]:
import math
covaff={}
for key,value in countryid.items():
    ry=[]
    for j in range(len(value)):
        if value[j]!=0:
            ry.append(value[j])
    amin, amax = min(ry), max(ry)
    if amax!=amin: 
        covaff[key] = ry
notafftd =(list(set(countryid.keys()) - set(covaff.keys())))
print ("Number of countries not yet affected by Coronavirus: ",len(notafftd))
print (*notafftd)
print ("Number of countries affected by Coronavirus: ",len(covaff.keys()))
print (*covaff.keys())

Number of countries not yet affected by Coronavirus:  20
Madagascar Syria Cape Verde Djibouti Gambia, The Haiti Holy See Uganda Chad Dominica Antigua and Barbuda Saint Vincent and the Grenadines East Timor Timor-Leste Eritrea Mozambique Kosovo Papua New Guinea Grenada Somalia
Number of countries affected by Coronavirus:  151
Afghanistan Albania Algeria Andorra Angola Argentina Armenia Australia Austria Azerbaijan Bahamas, The Bahrain Bangladesh Barbados Belarus Belgium Benin Bhutan Bolivia Bosnia and Herzegovina Brazil Brunei Bulgaria Burkina Faso Cabo Verde Cambodia Cameroon Canada Central African Republic Chile China Colombia Congo (Brazzaville) Congo (Kinshasa) Costa Rica Cote d'Ivoire Croatia Cruise Ship Cuba Cyprus Czechia Denmark Dominican Republic Ecuador Egypt El Salvador Equatorial Guinea Estonia Eswatini Ethiopia Fiji Finland France Gabon Georgia Germany Ghana Greece Guatemala Guinea Guyana Honduras Hungary Iceland India Indonesia Iran Iraq Ireland Israel Italy Jamaica Japan 

In [7]:
import json
import numpy as np
ncvdf=pd.DataFrame()


jdict={}
#jdict['Country']=[]
day=[]
for key,value in covaff.items():
    days=[];dvalue=[];dvalue2=[];ep=[];gf=[];log_ry=[];nlog_ry=[]
    for j in range(len(value)):
        log_ry.append(math.log10(value[j]))
    #plt.plot(log_ry,label=key)
    ##amin, amax = min(log_ry), max(log_ry) 
    ##print (amin,amax)
    ##for j, val in enumerate(log_ry):
    ##    nlog_ry.append((val-amin) / (amax-amin))
    for j in range(len(value)):
        if j>0:
            dvalue.append(((value[j]-value[j-1])))
    ndvalue = [float(i)/max(dvalue) for i in dvalue]  
    for j in range(len(dvalue)):
        if j >0:
            gf.append(dvalue[j]/dvalue[j-1])
            
    for j in range(len(value)):
        if j>0: ep.append(round((value[j]/value[j-1]),3))
    dvalue.insert(0,0)
    ndvalue.insert(0,0)
    ep.insert(0,0)
    gf.insert(0,0)
    gf.insert(0,0)
    jdict[key] = {"Cases":value,"Rate":dvalue,"NormRate":ndvalue,"Ep":ep,"GrowthFactor":gf}


<ipython-input-7-4c5ff4fa5f65>:24: RuntimeWarning: invalid value encountered in long_scalars
  gf.append(dvalue[j]/dvalue[j-1])
<ipython-input-7-4c5ff4fa5f65>:24: RuntimeWarning: divide by zero encountered in long_scalars
  gf.append(dvalue[j]/dvalue[j-1])


In [8]:
from ipywidgets import widgets
from ipywidgets import IntSlider as islide
from ipywidgets import FloatSlider,HBox,VBox
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib nbagg

def makeinteractive(Yaxis,Country):      
    impcnt=Country.split(",") 
    yax = list(jdict.keys())
    plt.style.use('ggplot')
    fig = plt.figure()
    ax = plt.subplot(111)

    ax.set_ylabel(Yaxis)
    ax.set_xlabel("Number of Days since the begining of COVID Horror")
                    
    for key,value in jdict.items():
        if key in impcnt:
            ax.plot(value[Yaxis],'-o',label=key)
            
    ax.legend(loc='center left', bbox_to_anchor=(0.95, 0.5),) 

In [9]:
widgets.interact_manual(makeinteractive,Yaxis=["Cases","Rate","NormRate","Ep","GrowthFactor"],Country="India,Italy,Australia,China,Iran,Pakistan,US,Spain,Germany,France,United Kingdom")

interactive(children=(Dropdown(description='Yaxis', options=('Cases', 'Rate', 'NormRate', 'Ep', 'GrowthFactor'…

<function __main__.makeinteractive(Yaxis, Country)>

In [240]:
#plt.xlabel('Number of days')
#plt.ylabel('Logarithmic scale of Confirmed Cases')
#plt.title('My COVID-19 Analysis')
plt_fig=plt.figure(title="Try")
for key,value in jdict.items():
    if key in ["India","Italy","Australia","China","Iran","Pakistan","US","Spain","Germany","France"]:
        plt.plot(value["GrowthFactor"],label=key)
plt.show()


IntSlider(value=60)